In [1]:
#!pip install --user psycopg2-binary 
#!pip install --user psycopg2 

In [139]:
import psycopg2
import json
import pandas as pd
import csv

In [140]:
with open('db_cred.json','r') as j:
    cred = json.load(j)

First, we create connection with PostgreSQL database.

In [141]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    database=cred["database"],
    user=cred["user"],
    password=cred["password"])

Connecting to the PostgreSQL database...


In [142]:
# create a cursor
cur = conn.cursor()

Then we create four tables for the dataset downloaded from https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks. The first table is authors, include id as primary key and author name.Table publishers has two columns, id as primary key and publisher name. Table books has seven columns, id as primary key, title, ISBN, publication_date in the value type of date, language code and two foreign keys: author_id and publisher_id, which reference to id in authors table and publishers table respectively. The last table is rating, which includes id as primary key, foreign key book_id references to id in books table, rating and ratings_count. Since the ratings_count could be large numbers, the value type need to be BIGINT.

Another thing to note here is that publication date from the dataset is in the format of "YYYY/MM/DD", so it should be indicated in the SQL command.

In [6]:
createCmd ="""
CREATE TABLE authors (
  id SERIAL PRIMARY KEY,
  name VARCHAR(800)
);

CREATE TABLE publishers (
  id SERIAL PRIMARY KEY,
  name VARCHAR(800)
);
CREATE TABLE books (
  id SERIAL PRIMARY KEY,
  title VARCHAR(800),
  isbn VARCHAR(20),
  publication_date DATE,
  language_code VARCHAR(15),
  authors_id BIGINT REFERENCES authors(id),
  publisher_id BIGINT REFERENCES publishers(id)
);

CREATE TABLE ratings (
  id SERIAL PRIMARY KEY,
  book_id BIGINT REFERENCES books(id),
  rating NUMERIC,
  ratings_count BIGINT
);

ALTER TABLE books
ALTER COLUMN publication_date TYPE DATE USING to_date(publication_date::text, 'YYYY/MM/DD');

"""
cur.execute(createCmd)
conn.commit()

Before importing data from csv file to our PostgreSQL database, we need to first clean the data. There are some missing values in the publication_date column, so I replaced them with the median publication date of the entire dataset using pandas functions. There are also some "?" and "'" symbols in the columns of author names, publisher names and books title due to file decoding issues, so I manually deleted them using txt file replace functionality and replace them with blank space. 

In [ ]:
# Load the dataset into a pandas dataframe
df = pd.read_csv("books.csv", encoding='windows-1252')

# Remove empty rows in the dataframe
df.dropna(inplace=True)

# Replace "?" with NaN values
df["publication_date"].replace("?", np.nan, inplace=True)
# Convert the publication_date column to datetime format
df["publication_date"] = pd.to_datetime(df["publication_date"], format="%m/%d/%Y", errors='coerce')

# Replace NaN values with the median publication date
median_publication_date = df["publication_date"].median()
df["publication_date"].fillna(median_publication_date, inplace=True)

# Save the cleaned dataframe to a new csv file
df.to_csv("books_cleaned.csv", index=False)

Next, I wrote SQL commands iteratively into txt files using Python. For easier management, I wrote four txt files, and each includes insert commands for one table. 

In [86]:

# Write the authors to a text file
with open("D:/CSGY_6513_Big_Data/books_cleaned.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)  # skip the headers
    # Open a text file for writing
    with open("authors.txt", "w", encoding='windows-1252') as out:
        num = 1  # Keep track of the line number
        # Loop through the rows in the CSV file
        for row in reader:
            authors = row[2]
            # Write the insert statement to the text file
            out.write("insert into authors (id, name) values ({}, '{}')\n".format(num, authors))
            num += 1

with open("D:/CSGY_6513_Big_Data/books_cleaned.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)  # skip the headers
    # Open a text file for writing
    with open("publishers.txt", "w", encoding='windows-1252') as out:
        num = 1  # Keep track of the line number
        # Loop through the rows in the CSV file
        for row in reader:
            publishers = row[11]
            # Write the insert statement to the text file
            out.write("insert into publishers (id, name) values ({}, '{}')\n".format(num, publishers))
            num += 1
        

# Open the CSV file
with open("D:/CSGY_6513_Big_Data/books_cleaned.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)  # skip the headers
    # Open a text file for writing
    with open("books.txt", "w", encoding='windows-1252') as out:
        num = 1  # Keep track of the line number
        # Loop through the rows in the CSV file
        for row in reader:
            title = row[1]  # Escape single quotes in the title
            isbn = row[4]
            publication_year = row[10]  # Take only the year from the publication_date
            language_code = row[6]
            # Write the insert statement to the text file
            out.write(
                "insert into books (id, title, isbn, publication_date, language_code, authors_id, publisher_id) values({}, '{}', '{}', '{}', '{}', {}, {});\n".format(
                    num, title, isbn, publication_year, language_code, num, num))
            num += 1

            
with open("D:/CSGY_6513_Big_Data/books_cleaned.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)  # skip the headers
    # Open a text file for writing
    with open("ratings.txt", "w", encoding='windows-1252') as out:
        num = 1  # Keep track of the line number
        # Loop through the rows in the CSV file
        for row in reader:
            rating = row[3]  # Escape single quotes in the title
            rcount = row[8]
            # Write the insert statement to the text file
            out.write("insert into ratings (id, book_id, rating, ratings_count) values ({}, {}, {}, {})\n".format(num, num, rating, rcount))
            num += 1


Insert data into authors table and print out the result.

In [91]:
with open('authors.txt', encoding='windows-1252') as f:
    for sql in f.readlines():
        cur.execute(sql)
conn.commit()

In [92]:
queryCmd = 'SELECT * from authors'    
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row[0], row[1])
    row = cur.fetchone()  

The number of rows:  11099
1 J.K. Rowling/Mary GrandPr
2 J.K. Rowling/Mary GrandPr
3 J.K. Rowling
4 J.K. Rowling/Mary GrandPr
5 J.K. Rowling/Mary GrandPr
6 W. Frederick Zimmerman
7 J.K. Rowling
8 Douglas Adams
9 Douglas Adams
10 Douglas Adams
11 Douglas Adams/Stephen Fry
12 Douglas Adams
13 Bill Bryson
14 Bill Bryson
15 Bill Bryson
16 Bill Bryson
17 Bill Bryson
18 Bill Bryson
19 Bill Bryson
20 Bill Bryson
21 Bill Bryson
22 J.R.R. Tolkien
23 J.R.R. Tolkien
24 J.R.R. Tolkien
25 J.R.R. Tolkien/Alan  Lee
26 Chris   Smith/Christopher  Lee/Richard Taylor
27 Jude Fisher
28 Dave Thomas/David Heinemeier Hansson/Leon Breedt/Mike Clark/Thomas  Fuchs/Andreas  Schwarz
29 Gary Paulsen
30 Donna Ickes/Edward Sciranko/Keith Vasconcelles
31 Gary Paulsen
32 Molly Hatchet
33 Dale Peck
34 Angela Knight/Sahara Kelly/Judy Mays/Marteeka Karland/Kate Douglas/Shelby Morgen/Lacey Savage/Kate Hill/Willa Okati
35 Delia Sherman
36 Patricia A. McKillip
37 Zilpha Keatley Snyder
38 Kate Horsley
39 Philippa Carr
40 Edw

Insert data into publishers table and print out the result.

In [93]:
with open('publishers.txt', encoding='windows-1252') as f:
    for sql in f.readlines():
        cur.execute(sql)
conn.commit()

In [94]:
queryCmd = 'SELECT * from publishers'    
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row[0], row[1])
    row = cur.fetchone()  

The number of rows:  11099
6067 
9204 DAW
1 Scholastic Inc.
2 Scholastic Inc.
3 Scholastic
4 Scholastic Inc.
5 Scholastic
6 Nimble Books
7 Scholastic
8 Gramercy Books
9 Del Rey Books
10 Crown
11 Random House Audio
12 Wings Books
13 Broadway Books
14 Broadway Books
15 Broadway Books
16 Broadway Books
17 Broadway Books
18 William Morrow Paperbacks
19 William Morrow Paperbacks
20 William Morrow Paperbacks
21 William Morrow Paperbacks
22 Ballantine Books
23 Houghton Mifflin Harcourt
24 Houghton Mifflin Harcourt
25 Houghton Mifflin Harcourt
26 Houghton Mifflin Harcourt
27 Houghton Mifflin Harcourt
28 Pragmatic Bookshelf
29 Atheneum Books for Young Readers: Richard Jackson Books
30 Teacher Created Resources
31 Delacorte Press
32 Cherry Lane Music Company
33 The New Press
34 Changeling Press
35 Viking Juvenile
36 Firebird
37 iUniverse
38 Shambhala
39 Ivy Books
40 Amistad
41 HarperAudio
42 Harper
43 FT Press
44 Archaia
45 Farrar  Straus and Giroux
46 Farrar Straus Giroux
47 Farrar  Straus and 

Insert data into books table and print out the result.

In [102]:
with open('books.txt', encoding='windows-1252') as f:
    for sql in f.readlines():
        cur.execute(sql)
conn.commit()

In [103]:
queryCmd = 'SELECT * from books'    
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row[0], row[1], row[2], row[3], row[4], row[5], row[6])
    row = cur.fetchone() 

The number of rows:  11099
1 Harry Potter and the Half-Blood Prince (Harry Potter  #6) 439785960 2006-09-16 eng 1 1
2 Harry Potter and the Order of the Phoenix (Harry Potter  #5) 439358078 2004-09-01 eng 2 2
3 Harry Potter and the Chamber of Secrets (Harry Potter  #2) 439554896 2003-11-01 eng 3 3
4 Harry Potter and the Prisoner of Azkaban (Harry Potter  #3) 043965548X 2004-05-01 eng 4 4
5 Harry Potter Boxed Set  Books 1-5 (Harry Potter  #1-5) 439682584 2004-09-13 eng 5 5
6 Unauthorized Harry Potter Book Seven News: "Half-Blood Prince" Analysis and Speculation 976540606 2005-04-26 en-US 6 6
7 Harry Potter Collection (Harry Potter  #1-6) 439827604 2005-09-12 eng 7 7
8 The Ultimate Hitchhiker s Guide: Five Complete Novels and One Story (Hitchhiker s Guide to the Galaxy  #1-5) 517226952 2005-11-01 eng 8 8
9 The Ultimate Hitchhiker s Guide to the Galaxy (Hitchhiker s Guide to the Galaxy  #1-5) 345453743 2002-04-30 eng 9 9
10 The Hitchhiker s Guide to the Galaxy (Hitchhiker s Guide to the Ga

10786 The Madman of Bergerac 141187263 2003-12-04 eng 10786 10786
10787 Dirty Snow 1590170431 2003-08-31 eng 10787 10787
10788 The Man Who Wasn t Maigret: A Portrait of Georges Simenon 156000598 1994-04-15 eng 10788 10788
10789 Maigret  New York (Maigret  #27) 2253142425 2002-12-04 fre 10789 10789
10790 The Engagement 1590172280 2007-03-06 en-US 10790 10790
10791 Mon Ami Maigret 2253142441 2003-04-23 fre 10791 10791
10792 Maigret and the Headless Corpse 156551446 1985-06-26 eng 10792 10792
10793 Maigret Loses His Temper (Maigret  #61) 156028476 2003-06-16 eng 10793 10793
10794 The Looking Glass Wars 803731531 2006-09-26 eng 10794 10794
10795 Rash 689868014 2006-06-01 eng 10795 10795
10796 Saints at the River 312424914 2005-07-01 eng 10796 10796
10797 The World Made Straight 312426607 2007-03-20 eng 10797 10797
10798 Eureka Mill 189188526X 2001-09-01 eng 10798 10798
10799 Chemistry and Other Stories 312425082 2000-09-05 eng 10799 10799
10800 Monster 64407314 2019-03-05 eng 10800 10800
1

Insert data into ratings table and print out the result.

In [143]:
with open('ratings.txt', encoding='windows-1252') as f:
    for sql in f.readlines():
        cur.execute(sql)
conn.commit()

In [144]:
queryCmd = 'SELECT * from ratings'    
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row[0], row[1], row[2], row[3])
    row = cur.fetchone()

The number of rows:  11099
1 1 4.57 2095690
2 2 4.49 2153167
3 3 4.42 6333
4 4 4.56 2339585
5 5 4.78 41428
6 6 3.74 19
7 7 4.73 28242
8 8 4.38 3628
9 9 4.38 249558
10 10 4.22 4930
11 11 4.22 1266
12 12 4.38 2877
13 13 4.21 248558
14 14 3.44 7270
15 15 3.87 2088
16 16 4.07 72451
17 17 3.9 49240
18 18 3.83 45712
19 19 3.86 48701
20 20 3.91 80609
21 21 3.93 28489
22 22 4.59 101233
23 23 4.5 1710
24 24 4.36 2128944
25 25 4.5 1618
26 26 4.53 19822
27 27 4.5 359
28 28 3.84 1430
29 29 3.72 270244
30 30 4 36
31 31 3.88 2067
32 32 4.33 6
33 33 3.45 99
34 34 3.76 167
35 35 3.6 978
36 36 4.06 4454
37 37 4.17 1176
38 38 3.55 301
39 39 3.98 345
40 40 3.83 29686
41 41 3.83 55
42 42 3.83 22
43 43 3.83 1456
44 44 4.13 52
45 45 4.42 562
46 46 3.95 1630
47 47 4.17 268
48 48 4.24 3498
49 49 4.34 3115
50 50 4.22 5704
51 51 3.92 698
52 52 4.23 542
53 53 4.23 1341
54 54 3.75 1423
55 55 3.84 2766
56 56 3.94 31
57 57 3.77 62
58 58 4.46 860
59 59 3.68 77
60 60 3.99 153317
61 61 3.25 55
62 62 4.46 34
63 63 3.89

Question: What are the oldest 5 books in this dataset?

The following query retrieves book titles and publication dates, then sort them according to the publication date and keep the first five rows.

In [145]:
queryCmd1 = '''
SELECT title, publication_date
FROM books
ORDER BY publication_date
LIMIT 5;
'''
cur.execute(queryCmd1) 

rows = cur.fetchall()
for row in rows:
    print(row)

('Consider the Lilies', datetime.date(1900, 1, 1))
('On Duties (De Officiis)', datetime.date(1913, 1, 1))
('Agricola / Germania / Dialogue on Oratory', datetime.date(1914, 1, 1))
('History of the Peloponnesian War: Bk. 1-2', datetime.date(1919, 1, 1))
('History of the Peloponnesian War: Bk. 5-6', datetime.date(1921, 1, 1))


Question: Which 5 language codes have the highest rating average?

The following query retreives language_code and calculate average rating of each language code, then sort the result by the average rating in the descending order and keep the first five rows.

In [146]:
queryCmd2 = '''
SELECT language_code, AVG(rating) AS avg_rating
FROM books
JOIN ratings ON books.id = ratings.book_id
GROUP BY language_code
ORDER BY avg_rating DESC
LIMIT 5;
'''
cur.execute(queryCmd2)

rows2 = cur.fetchall()
for r in rows2:
    print(r)

('wel', Decimal('5.0000000000000000'))
('gla', Decimal('4.4700000000000000'))
('tur', Decimal('4.4200000000000000'))
('zho', Decimal('4.3900000000000000'))
('ale', Decimal('4.3600000000000000'))


Question: Which 5 authors have the highest number of ratings?

The following query retreives authors name and sum up the number of ratings in the group of authors name, then sort the result in the descending order, and keeps the first five rows.

In [147]:
queryCmd3 = '''
SELECT name, SUM(ratings_count) AS num_ratings
FROM authors
JOIN books ON authors.id = books.authors_id
JOIN ratings ON books.id = ratings.book_id
GROUP BY name
ORDER BY num_ratings DESC
LIMIT 5;
'''
cur.execute(queryCmd3) 

rows = cur.fetchall()
for row in rows:
    print(row)

('J.K. Rowling/Mary GrandPr', Decimal('8923980'))
('J.R.R. Tolkien', Decimal('4776638'))
('Stephenie Meyer', Decimal('4597666'))
('Dan Brown', Decimal('4135380'))
('Nicholas Sparks', Decimal('3048149'))


Question: Which publishers publish in the highest number of different languages?

The following query retreives the publishers name and counts the number of different language_code for each publisher, and sort the result in the descending order, and keeps the first five rows.

In [148]:
queryCmd4 = '''
SELECT name, COUNT(DISTINCT language_code) AS num_languages
FROM publishers
JOIN books ON publishers.id = books.publisher_id
GROUP BY name
ORDER BY num_languages DESC
LIMIT 5;
'''
cur.execute(queryCmd4) 

rows = cur.fetchall()
for row in rows:
    print(row)

('Oxford University Press  USA', 6)
('Dover Publications', 5)
('Mariner Books', 5)
('HarperOne', 4)
('Cambridge University Press', 4)
